In [4]:
pip install functions-framework

Note: you may need to restart the kernel to use updated packages.


In [5]:
# For adding  weathers information in weathers table in sql database
import functions_framework
import pandas as pd
import requests
from datetime import datetime, timedelta
from pytz import timezone
from keys import API_key_weather, sql_password


@functions_framework.http
# Read cities data from database
# get weather data from weather api for the cicites in databse
# write weather data retrived above in weather table of database
def retrive_save_weather_info(request):
    connection_string = create_connection_string()
    ret_cities_df = fetch_cities_data(connection_string)
    weather_df = fetch_weather_data(ret_cities_df)
    save_weather_data(weather_df, connection_string)
    return "Weather information retrived and saved in database"

#function to create connection string to make connection with Database
def create_connection_string():
  schema = "gans"
  host = "35.241.135.102"
  user = "root"
  password = sql_password
  port = 3306
  return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

# To read cities info from cities table
def fetch_cities_data(connection_string):
  return pd.read_sql("cities", con=connection_string)

# to get weather data from weather API for cities in the cities 
def fetch_weather_data(cities_df):
    
  berlin_timezone = timezone('Europe/Berlin')
  weather_items = []
    
  for _, city in cities_df.iterrows():
      latitude = city["latitude"]
      longitude = city["longitude"]
      city_id = city["city_id"]

      url = (f"https://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&appid={API_key_weather}&units=metric")
      response = requests.get(url)
      if response.status_code ==200:
          weather_data = response.json()

          retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

          for item in weather_data["list"]:
              weather_item = {
                  "city_id": city_id,
                  "forecast_time": item.get("dt_txt"),
                  "temperature": item["main"].get("temp"),
                  "forecast": item["weather"][0].get("main"),
                  "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
                  "snow_in_last_3h": item.get("snow", {}).get("3h", 0),
                  "visibility":item.get("visibility"),
                  "wind_speed": item["wind"].get("speed"),
                  "data_retrieved_at": retrieval_time
              }
              weather_items.append(weather_item)

  weather_df = pd.DataFrame(weather_items)
  weather_df["forecast_time"] = pd.to_datetime(weather_df["forecast_time"])
  weather_df["data_retrieved_at"] = pd.to_datetime(weather_df["data_retrieved_at"])

  return weather_df

# Function to Save weather data
def save_weather_data(weather_df, connection_string):
  weather_df.to_sql('weathers',
                    if_exists='append',
                    con=connection_string,
                    index=False)



In [7]:
from flask import Request
import json

request_data = {}
request = Request.from_values(data=json.dumps(request_data))

# Call the function
response = retrive_save_weather_info(request)
print(response)

Weather information retrived and saved in database
